In [355]:
import pandas as pd
import re

reference_fields_df = pd.read_csv("reference_fileds.csv")
reference_securities_df = pd.read_csv("reference_securities.csv")

In [350]:
# Overview: 
# - Pulls all the column names from the corp_pfd.dif file and saves them in a set of strings dif_columns

#Read the dif file as a string
with open("corp_pfd.dif", "r") as file:    
    full_dif_string = file.read() 

#Grab all column names between the START/END-OF-FIELDS
dif_columns =  re.search("START-OF-FIELDS(.*)END-OF-FIELDS", full_dif_string, re.DOTALL)
dif_columns= dif_columns.group(1)

#Split based on new line to seperate every column name
dif_columns= dif_columns.split("\n")

#Filter out all headers containing '#' and blank lines
dif_columns = [column for column in dif_columns if '#' not in column and column.strip()]

In [351]:
# Overview: 
# - Creates DataFrame 

#Find all the columns in dif_column_name that are also in reference_feilds
ref_fields_set = set(reference_fields_df['field'])
res = [col for col in dif_columns if col in ref_fields_set] 

#creates an empty data frame with the appropriate column headers
DataFrame= pd.DataFrame(columns=res)

#This loop adds all the data from reference_securities_csv to DataFrame in the approriate columns
for column in reference_securities_df.columns:
    DataFrame[column.upper()]= reference_securities_df[column]

In [352]:
#Overview: 
# - Formats the input dif file into a dataframe

#Grab all the data in the dif file between the START/END-OF-DATA
data_dif_string =  re.search("START-OF-DATA(.*)END-OF-DATA", full_dif_string, re.DOTALL)
data_dif_string= data_dif_string.group(1)

#remove all leading and trailing spaces/new lines to prevent blank items in the list after splitting
data_dif_string= data_dif_string.strip() 

#split based on new line to seperate every row
seperated_data= data_dif_string.split("\n")

#iterate every row and split based on '|' delimiter, then add each list of row values to all_rows
all_rows=[]
for row in seperated_data:
    sep_rows = row.split("|")
    all_rows.append(sep_rows[:len(sep_rows)-1])  #each row ends with '|' so spliting by '|' leaves a trailing ' ' 

input_data= pd.DataFrame(all_rows,columns=dif_columns)

In [353]:
# Overview:
# - Creates new_securities.csv

#find all ids in input dif file not in reference_securities.csv
ref_securities_ids= set(reference_securities_df['id_bb_global'])
input_ids = set(input_data["ID_BB_GLOBAL"])
new_ids= input_ids - ref_securities_ids

new_securities_rows=[]

#creates a list of rows to add to the new_securities_df 
for id in new_ids:
    first_index = input_data.index[input_data['ID_BB_GLOBAL'] == id][0]
    row=[]
    for column in reference_securities_df.columns:
        new_val= input_data.at[first_index, column.upper()]
        row.append(new_val)
    new_securities_rows.append(row)

new_securities_df= pd.DataFrame(new_securities_rows,columns=reference_securities_df.columns)

# new_securities_df.to_csv("/Users/dominiclaiosa/Downloads/Resume/Cantor Assignment/new_securities.csv", index=False, encoding="utf-8")

In [354]:
#Overview:
# - Creates Security_data.csv

security_data= pd.DataFrame(columns=res)

for col in res:
    security_data[col]=input_data[col]

#unpivots the data so for every ID there is a seperate row for each feild it has that contains a value
security_data_unpivoted = security_data.melt(
    id_vars=['ID_BB_GLOBAL'],  
    var_name="FIELD",          
    value_name="VALUE"         
)

#Pulls the date and time from the dif file- 'TIMEFINISHED=Mon May  3 19:50:37 EDT 2021' 
match = re.search(r"TIMEFINISHED=(.*)", full_dif_string)
match= match.group(1).strip()
timefinished_cleaned = re.sub(r"\s[A-Z]{3}\s", " ", match)  # Removes " EDT "
tstamp = pd.to_datetime(timefinished_cleaned).tz_localize("America/New_York")

#assigns source and tstamp vals
security_data_unpivoted['SOURCE'] = "corp_pfd.dif"
security_data_unpivoted['TSTAMP'] = tstamp

#removes all rows with blank entries from the unpivoted data frame
security_data_unpivoted = security_data_unpivoted.dropna(subset=["VALUE"])
security_data_unpivoted = security_data_unpivoted[security_data_unpivoted["VALUE"].str.strip() != ""]

#sorts the rows so all the same ID rows appear next to eachother
security_data_unpivoted = security_data_unpivoted.sort_values(by=['ID_BB_GLOBAL', 'FIELD']).reset_index(drop=True)


security_data_unpivoted.to_csv("/Users/dominiclaiosa/Downloads/Resume/Cantor Assignment/securities_data.csv", index=False, encoding="utf-8")